# Calibration of a dysfunctional case without considering events

This notebook shows a calibration of a dysfunctional case (a broken fan) based on model without considering the event.

The calibration methodology is still the same as a [basic calibration on transient data](calibration_transient_basics.ipynb): the heat exchange coefficient adder is computed to fit the temperature increase during a time step.

### Model setup

In [ ]:
from cpu.systems import CPUSystem

cpu = CPUSystem.load("data/cpu_ref.json")

### Calibration setup

In [ ]:
import numpy as np
import pandas as pd

from cosapp.drivers import NonLinearSolver, EulerExplicit
from cosapp.drivers.time.scenario import Interpolator
from cosapp.recorders import DataFrameRecorder

time_driver = cpu.add_driver(EulerExplicit())
solver = time_driver.add_child(NonLinearSolver("solver", max_iter=10, factor=1.0, tol=1e-6))

# load the operation data, including fan breakage event
data = pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv")
next_T = data["T_cpu"][1:]

# define a calibration methodology
solver.add_equation("cpu.expected_next_T == cpu.next_T").add_unknown("exchanger.h_adder")

time_driver.time_interval = (0, 29)
time_driver.dt = 1.0

# define a simulation scenario using the operating conditions
time_driver.set_scenario(
    init={"T_cpu": 10.0},
    values={
        "cpu.expected_next_T": Interpolator(np.stack([np.linspace(0, 29, 30), next_T], axis=1)),
        "fan.T_air": Interpolator(np.stack([data.index, data["fan.T_air"]], axis=1)),
        "cpu.usage": Interpolator(np.stack([data.index, data["cpu.usage"]], axis=1)),
    },
)

rec = time_driver.add_recorder(DataFrameRecorder(includes=["*"], hold=False), period=1.0)

cpu.run_drivers()
rec.data.to_csv("data/calibrated_simulation_broken1.csv")

In [ ]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "operation": pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv"),
        "calibrated": pd.read_csv("data/calibrated_simulation_broken1.csv"),
    },
    [
        [("time", "fan.T_air"), ("time", "cpu.usage")],
        [("time", "fan.tension"), ("time", "T_cpu")],
        [("time", "exchanger.h_adder")],
    ],
    width=1000,
    height=600,
)

### Analysis

The results are completely wrong because we calibrated a fully working model on a dataset including a breakage!

In [ ]:
np.median(rec.data["exchanger.h_adder"])